<a href="https://colab.research.google.com/github/nalinapk/ProgramminginGIS-II/blob/main/Practical_Assignment6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import geemap.geemap as map

In [2]:
import ee

In [3]:
ee.Authenticate()

In [4]:
ee.Initialize(project = 'ee-nalinapk17')

In [5]:
m=map.Map()
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [37]:
India_states = ee.FeatureCollection('projects/ee-nalinapk17/assets/India_states')
Telangana = India_states.filter(ee.Filter.eq('STATE','TELANG>NA'))
m.addLayer(Telangana,{},'Telangana')
m

Map(bottom=7783.0, center=[15.390135715305217, 77.38769531250001], controls=(WidgetControl(options=['position'…

<IPython.core.display.Javascript object>

style = {'color': '0000007f', 'width': 2, 'lineType': 'solid', 'fillColor': '000000a8'}


In [38]:
region = ee.Geometry.Polygon(
    [[[77.0, 15.8],  # Bottom-left
      [77.0, 19.9],  # Top-left
      [81.9, 19.9],  # Top-right
      [81.9, 15.8],  # Bottom-right
      [77.0, 15.8]]], # Closing the polygon
    None,
    False)

In [39]:
NDVI = ee.ImageCollection("MODIS/061/MOD13A2").select('NDVI')
m

Map(bottom=7585.0, center=[19.539084135509334, 87.07763671875001], controls=(WidgetControl(options=['position'…

In [40]:
NDVI_Telangana = NDVI.map(lambda img: img.set('doy',
ee.Date(img.get('system:time_start')).getRelative('day', 'year')));

In [41]:
distinctDOY = NDVI_Telangana.filterDate('2020-02-18', '2021-02-18');

In [42]:
filter = ee.Filter.equals(leftField= 'doy', rightField= 'doy')
join = ee.Join.saveAll('doy_matches')
joincol =ee.ImageCollection(join.apply(distinctDOY, NDVI_Telangana, filter))

In [53]:
def add_date_from(img):
  doy = ee.Number(img.get('doy'))

In [54]:
composite = joincol.map(lambda img: ee.ImageCollection.fromImages(img.get('doy_matches')).reduce(ee.Reducer.mean()).set('doy',ee.Number(img.get('doy'))))

In [55]:
Vis_para = {
  'min': 2000,
  'max': 10000,
  'palette': [
    'ffffff', 'ce7e45', 'df923d', 'f1b555', 'fcd163', '99b718', '74a901',
    '66a000', '529400', '3e8601', '207401', '056201', '004c00', '023b01',
    '012e01', '011d01', '011301'
  ],
};

In [56]:
rgbvis= composite.map(lambda img: img.visualize(bands = ['NDVI_mean'],**Vis_para).clip(Telangana))

In [77]:
gifParams = {
  'region': region,
  'dimensions': 600,
  'crs': 'EPSG:4326',
  'framesPerSecond': 120,
  'format':'gif',
  'addDate': 'yyyy-mm-dd'
}

In [78]:
gif_url = rgbvis.getVideoThumbURL(gifParams)

In [79]:
print(rgbvis.getVideoThumbURL(gifParams))

https://earthengine.googleapis.com/v1/projects/ee-nalinapk17/videoThumbnails/e53d39e56a8f27f41edc89984226b9d6-1e8f0c91ee197a8bce8043046a0a2b6c:getPixels
